- https://github.com/huggingface/trl/issues/2809#issuecomment-2646363608

$$
f(x)=\max\left(-\left(\frac x5\right)^2+x+1, 0\right)
$$

In [4]:
def get_value(x: float) -> float:
    """
    Get the value of the function at x.

    Args:
        x: The input value.

    Returns:
        The value of the function at x.
    """
    return max(-((x / 5) ** 2) + x + 1, 0.0)

In [7]:
from transformers import AutoTokenizer
from datasets import Dataset

In [2]:
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-0.5B-Instruct', padding_side="left")

In [6]:
print(tokenizer.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are Qwen, created by Alibaba C

In [9]:
dataset = Dataset.from_list(
    [{"prompt": [{"role": "user", "content": "Call the function get_value with any value."}]}] * 200
)
example = dataset[0]

In [12]:
prompt = tokenizer.apply_chat_template(example['prompt'], tools=[get_value], tokenize=False, add_generation_prompt=True)

In [13]:
print(prompt)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_value", "description": "Get the value of the function at x.", "parameters": {"type": "object", "properties": {"x": {"type": "number", "description": "The input value."}}, "required": ["x"]}, "return": {"type": "number", "description": "The value of the function at x."}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
Call the function get_value with any value.<|im_end|>
<|im_start|>assistant



In [14]:
print(tokenizer.apply_chat_template(example['prompt'], tools=[get_value], tokenize=False))

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_value", "description": "Get the value of the function at x.", "parameters": {"type": "object", "properties": {"x": {"type": "number", "description": "The input value."}}, "required": ["x"]}, "return": {"type": "number", "description": "The value of the function at x."}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
Call the function get_value with any value.<|im_end|>

